# Financial Analysis with Data Science & Machine Learning - Part 1
## Data Loading and Exploration

This notebook performs the initial data loading and exploratory analysis of financial indicators for US stocks.

## 1. Setup and Data Loading

In [ ]:
# Import necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import kagglehub

# Set visualization style
plt.style.use('ggplot')
sns.set_theme(style="whitegrid")

# Display all columns and rows
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
# Download the dataset using kagglehub
path = kagglehub.dataset_download("cnic92/200-financial-indicators-of-us-stocks-20142018")
print("Path to dataset files:", path)

In [ ]:
# List all files in the downloaded dataset
data_files = os.listdir(path)
print("Files in the dataset:")
for file in data_files:
    print(f"- {file}")

In [ ]:
# Load the dataset (assuming CSV format, adjust if needed)
# Try to find the main data file
csv_files = [f for f in data_files if f.endswith('.csv')]
if csv_files:
    main_file = os.path.join(path, csv_files[0])
    data = pd.read_csv(main_file)
    print(f"Loaded data from {csv_files[0]}")
else:
    print("No CSV files found. Please check the dataset structure.")
    # Alternative: try to directly access a known file if the structure is known
    # data = pd.read_csv(os.path.join(path, 'financial_data.csv'))

## 2. Initial Data Exploration

In [ ]:
# Display basic information about the dataset
print("Dataset shape:", data.shape)
print("\nFirst 5 rows of the dataset:")
data.head()

In [ ]:
# Data types and missing values
print("\nData types:")
data.info()

In [ ]:
# Check for missing values
missing_values = data.isnull().sum()
missing_percent = (missing_values / len(data)) * 100

missing_df = pd.DataFrame({
    'Missing Values': missing_values,
    'Percentage': missing_percent
})

# Display columns with missing values
missing_df = missing_df[missing_df['Missing Values'] > 0].sort_values('Missing Values', ascending=False)
if not missing_df.empty:
    print("\nColumns with missing values:")
    missing_df
else:
    print("\nNo missing values found in the dataset.")

In [ ]:
# Statistical summary of numerical features
print("\nStatistical summary of numerical features:")
data.describe()

In [ ]:
# Count of companies by sector
if 'Sector' in data.columns:
    plt.figure(figsize=(14, 8))
    sector_counts = data['Sector'].value_counts()
    sns.barplot(x=sector_counts.values, y=sector_counts.index)
    plt.title('Number of Companies by Sector')
    plt.xlabel('Count')
    plt.tight_layout()
    plt.show()
    
    print("\nCompany count by sector:")
    print(sector_counts)

## 3. Handling Missing Values and Data Cleaning

In [ ]:
# Function to identify and handle outliers
def handle_outliers(df, column, method='cap', threshold=3):
    """Handle outliers in a dataframe column
    
    Parameters:
    -----------
    df : pandas DataFrame
        The dataframe containing the column with outliers
    column : str
        The column name to check for outliers
    method : str, optional (default='cap')
        The method to handle outliers ('cap' or 'remove')
    threshold : float, optional (default=3)
        The z-score threshold to identify outliers
        
    Returns:
    --------
    pandas DataFrame
        Dataframe with handled outliers
    """
    # Create a copy of the dataframe
    df_clean = df.copy()
    
    # Skip non-numeric columns
    if not np.issubdtype(df_clean[column].dtype, np.number):
        print(f"Column '{column}' is not numeric. Skipping.")
        return df_clean
    
    # Calculate z-scores
    z_scores = np.abs((df_clean[column] - df_clean[column].mean()) / df_clean[column].std())
    outliers = z_scores > threshold
    
    # Handle outliers based on the selected method
    if method == 'cap':
        # Cap outliers at the threshold value
        upper_bound = df_clean[column].mean() + threshold * df_clean[column].std()
        lower_bound = df_clean[column].mean() - threshold * df_clean[column].std()
        
        # Cap upper and lower bounds
        df_clean.loc[df_clean[column] > upper_bound, column] = upper_bound
        df_clean.loc[df_clean[column] < lower_bound, column] = lower_bound
        
        print(f"Capped {outliers.sum()} outliers in '{column}'")
    elif method == 'remove':
        # Remove rows with outliers
        df_clean = df_clean[~outliers]
        print(f"Removed {outliers.sum()} outliers from '{column}'")
    
    return df_clean

In [ ]:
# Clean the data - handle missing values
# Strategy depends on the actual data, but here's a generic approach
data_clean = data.copy()

# 1. Drop columns with too many missing values (e.g., >50%)
high_missing_cols = missing_df[missing_df['Percentage'] > 50].index.tolist()
if high_missing_cols:
    print(f"Dropping columns with >50% missing values: {high_missing_cols}")
    data_clean = data_clean.drop(columns=high_missing_cols)

# 2. Impute remaining missing values - numerical columns with median, categorical with mode
numeric_cols = data_clean.select_dtypes(include=np.number).columns
categorical_cols = data_clean.select_dtypes(exclude=np.number).columns

# For numeric columns: use median (more robust to outliers than mean)
for col in numeric_cols:
    if data_clean[col].isnull().sum() > 0:
        median_val = data_clean[col].median()
        data_clean[col].fillna(median_val, inplace=True)
        print(f"Filled missing values in '{col}' with median: {median_val:.2f}")

# For categorical columns: use mode (most frequent value)
for col in categorical_cols:
    if data_clean[col].isnull().sum() > 0:
        mode_val = data_clean[col].mode()[0]
        data_clean[col].fillna(mode_val, inplace=True)
        print(f"Filled missing values in '{col}' with mode: {mode_val}")

In [ ]:
# Handle outliers in key financial metrics
key_metrics = ['Total Revenue', 'Gross Profit', 'Operating Income', 'Net Income', 
               'Total Assets', 'Total Liabilities', 'Equity', 'Cash and Cash Equivalents',
               'Earnings Per Share (EPS)', 'Price-to-Earnings Ratio (P/E)', 'Dividend Yield', 
               'Market Capitalization']

# Handle outliers for each key metric that exists in our data
for metric in key_metrics:
    if metric in data_clean.columns:
        data_clean = handle_outliers(data_clean, metric, method='cap', threshold=3)

In [ ]:
# Save cleaned data for next notebooks
data_clean.to_csv('cleaned_financial_data.csv', index=False)
print("Saved cleaned data to 'cleaned_financial_data.csv'")

## 4. Data Profile Summary

In [ ]:
# Display summary of the cleaned dataset
print("Cleaned dataset shape:", data_clean.shape)
print(f"Original dataset had {data.shape[0]} rows and {data.shape[1]} columns")
print(f"Cleaned dataset has {data_clean.shape[0]} rows and {data_clean.shape[1]} columns")

# Check for any remaining missing values
remaining_missing = data_clean.isnull().sum().sum()
print(f"Remaining missing values: {remaining_missing}")

## Next Steps

In the next notebook, we will:
1. Calculate financial ratios
2. Perform more detailed exploratory data analysis
3. Create visualizations to understand the relationships between financial variables